# Under-determined Least Squares


In [19]:
using LinearAlgebra
function underLS(A,b; ϵ = 1e-14)
    """
    Solves an underdetermined linear system given
    the coefficient matrix A and the constant
    vector b. Returns the least norm solution.
    """
    n, m = size(A)
    s = min(n,m)
    F = qr(A, Val(true))
    
    #Compute rank approximation r
    Rtrm = F.R[1:s,1:s]  # trim R, it cannot be larger than min(rows,cols)
    r = maximum(findall(abs.(diag(Rtrm)) .>= ϵ))  # check if it is smaller
    l = m - r  # dimension of the remainder
    
    #Generate R and S
    R, S = F.R[1:r,1:r], F.R[1:r,r+1:end] # partition R
    d, P = R\(F.Q'*b)[1:r], R\S   # solve the partitioned parts
    z2 = consistentLS(P'*P + Matrix(I,l,l), P'*d)
    z1 = d - P*z2
    return F.P*vcat(z1,z2)
end

underLS (generic function with 1 method)

## Step through an example

### Generate a random problem
Including it's solution.

In [20]:
n,m = 4,6
A = rand(n,m)
x = rand(m)  # a solution
display(x)
display(norm(x))
b = A*x

6-element Array{Float64,1}:
 0.26813819034362996
 0.3949843824463437 
 0.5505346412945606 
 0.27541289197084073
 0.8113147053166976 
 0.30453913484168216

1.1652583574210438

4-element Array{Float64,1}:
 1.3936755236736698
 1.1794543362485006
 1.787046909380469 
 1.3902133329570854

### Initial QR solution
After some setting up some book-keeping variables.

In [21]:
ϵ = 1e-14
n, m = size(A)
s = min(n,m)
F = qr(A, Val(true))
display(F.Q)
display(F.R)  # R is augmented

4×4 LinearAlgebra.QRPackedQ{Float64,Array{Float64,2}}:
 -0.583062   0.397354   0.606697    0.366152
 -0.66993   -0.344704  -0.0214374  -0.657201
 -0.095447   0.83158   -0.440527   -0.324501
 -0.44958   -0.178225  -0.661358    0.57334 

4×6 Array{Float64,2}:
 -1.29186  -0.844453  -0.76269   -0.990148   -0.976032    -0.934128
  0.0       0.971375   0.520253  -0.0172961   0.544682     0.377195
  0.0       0.0       -0.733662  -0.626253   -0.379495    -0.529199
  0.0       0.0        0.0       -0.370814    0.00707627   0.232929

### Find the rank of R

In [22]:
Rtrm = F.R[1:s,1:s]  # trim R, it cannot be larger than min(rows,cols)
r = maximum(findall(abs.(diag(Rtrm)) .>= ϵ))  # check if it is smaller
l = m - r  # dimension of the remainder
println("The rank of R = $r")
println("There is/are $l remaining columns")

The rank of R = 4
There is/are 2 remaining columns


### Generate R and S


In [23]:
R, S = F.R[1:r,1:r], F.R[1:r,r+1:end] # partition R
display(R)  # this can now be used to backsub
display(S)  # what's left

4×4 Array{Float64,2}:
 -1.29186  -0.844453  -0.76269   -0.990148 
  0.0       0.971375   0.520253  -0.0172961
  0.0       0.0       -0.733662  -0.626253 
  0.0       0.0        0.0       -0.370814 

4×2 Array{Float64,2}:
 -0.976032    -0.934128
  0.544682     0.377195
 -0.379495    -0.529199
  0.00707627   0.232929

In [24]:
include("../jl/backsub.jl")
d = backsub(R, F.Q'*b[1:r])
display(d)
P = R\S   # solve the partitioned parts

4-element Array{Float64,1}:
 0.5601449271149894
 0.840320490783169 
 1.0984645562469506
 0.1285689278047092

4×2 Array{Float64,2}:
  0.275632    0.655862
  0.274632   -0.296375
  0.533551    1.25751 
 -0.0190831  -0.628156

## Work with the remaining columns to find a minimum norm solution

In [25]:
display(P'*P + Matrix(I,l,l))
display(P'*d)

2×2 Array{Float64,2}:
 1.43644   0.782314
 0.782314  3.4939  

2-element Array{Float64,1}:
 0.9688063230484489
 1.4188925616420773

In [26]:
include("../jl/consistentLS_dh.jl")
z2 = consistentLS(P'*P + Matrix(I,l,l), P'*d)
display(z2)
z1 = d - P*z2
z = F.P*vcat(z1,z2)
display(z)
norm(z) # less than or equal to the norm(x)

2-element Array{Float64,1}:
 0.516228729474596  
 0.29051802624617584

6-element Array{Float64,1}:
 0.29051802624617584
 0.516228729474596  
 0.4577017771803569 
 0.22731595104375613
 0.7846499946605625 
 0.3209108850596343 

1.1535665101815724

In [27]:
underLS(A,b)

6-element Array{Float64,1}:
 0.29051802624617584
 0.516228729474596  
 0.45770177718035665
 0.22731595104375624
 0.7846499946605625 
 0.3209108850596343 

In [29]:
y = underLS(A,b)
A*y-b  # all zero

4-element Array{Float64,1}:
  0.0                  
 -2.220446049250313e-16
  0.0                  
 -2.220446049250313e-16

In [30]:
# Example 2

printstyled(
"EXAMPLE 2: Fat Matrix\n", color=:red)
n, m = 4, 10
A = rand(n,m)
b = rand(n)
println("A is an $n by $m matrix")
println("Error between underLS and 'truth':
    $(norm(underLS(A,b) - A\b))")

EXAMPLE 2: Fat Matrix
A is an 4 by 10 matrix
Error between underLS and 'truth':
    3.286019147154656e-16


In [31]:
# Example 3

printstyled(
"EXAMPLE 3: Repeating Tall Matrix\n", color=:red)
n, m = 5,10
A = rand(n,m)
b = rand(n)
A = vcat(A,A,A,A)
b = vcat(b,b,b,b)
println("A is an $(size(A,1)) by $(size(A,2)) matrix.")
println("Error between underLS and 'truth':
    $(norm(underLS(A,b) - A\b))")

EXAMPLE 3: Repeating Tall Matrix
A is an 20 by 10 matrix.
Error between underLS and 'truth':
    1.852827602846592e-16
